# Predictor

In [49]:
import os
from datetime import datetime, timedelta

import keras
import numpy as np
import pandas as pd

In [ ]:
# os.chdir("src") #used to reset to original working directory

In [3]:
os.chdir(
    "../"
)  # Used to go out into the cmpe-351-group-1 folder to access the data without changing directories multiple times

In [324]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return t.replace(
        second=0, microsecond=0, minute=0, hour=t.hour
    ) + timedelta(hours=t.minute // 30)

In [325]:
def normalize(df, column):
    m = df[column].mean()
    s = df[column].std()
    df[column] = df[column].apply(lambda x: (x - m) / s)
    return df[column]

In [326]:
##Dates is an array of tuples consisting of month and day as numbers
def get_dataset(crypto, start_month, start_day, end_month, end_day):
    twitter_df = pd.read_csv(f"data/processed/twitter/{crypto}_sentiment.csv")
    try:
        crypt_prices = pd.read_csv(
            f"data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv"
        )
    except:
        crypt_prices = pd.read_csv(
            f"data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv"
        )
    crypt_prices["time"] = pd.to_datetime(crypt_prices["time"], unit="s")
    price_dict = crypt_prices.set_index("time")["open"].to_dict()
    twitter_df["created_at"] = twitter_df["created_at"].apply(
        lambda x: pd.to_datetime(x).tz_localize(None)
    )
    twitter_df["created_at"] = twitter_df["created_at"].apply(
        lambda x: hour_rounder(x)
    )
    twitter_df["future_date"] = twitter_df["created_at"].apply(
        lambda x: x + timedelta(hours=23)
    )

    twitter_df = twitter_df[
        twitter_df["future_date"].isin(crypt_prices["time"])
    ]  ##Filter out dates for which we don't have price data

    # print(len(twitter_df))

    twitter_df = twitter_df[
        twitter_df["created_at"].isin(crypt_prices["time"])
    ]  ##Filter out dates for which we don't have price data

    twitter_df["price"] = twitter_df["created_at"].apply(
        lambda x: price_dict[x]
    )
    for i in range(1, 24):
        twitter_df[f"price_{i}hours"] = twitter_df["created_at"].apply(
            lambda x: price_dict[x + timedelta(hours=i)]
        )
    twitter_df["retweet_count"] = normalize(twitter_df, "retweet_count")
    twitter_df["favourite_count"] = normalize(twitter_df, "retweet_count")
    twitter_df["followers_count"] = normalize(twitter_df, "retweet_count")
    twitter_df["authority"] = twitter_df[
        ["retweet_count", "favourite_count", "followers_count"]
    ].sum(axis=1)

    X = twitter_df[["authority", "vader_sentiment_compound", "price"]]
    y = twitter_df[[f"price_{i}hours" for i in range(1, 24)]]
    return twitter_df, X, y

In [378]:
def make_model(n=3):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(8, input_shape=(n, 1)))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(23))
    model.compile(
        loss="mean_squared_error",
        optimizer="adam",
        metrics=["mean_squared_error"],
    )
    return model

In [328]:
def estimate_price(model, date, df):
    date_23_hours = date - timedelta(hours=23)
    date_1_hours = date - timedelta(hours=1)
    df_useful = df[
        (df["created_at"] <= date_1_hours)
        & (df["created_at"] >= date_23_hours)
    ]
    if len(df_useful) == 0:
        return ""
    else:
        df_useful["idx"] = df_useful["created_at"].apply(
            lambda x: round((date - x).seconds / 60 / 60)
        )
        X = df_useful[["authority", "vader_sentiment_compound", "price"]]
        y_hat = model.predict_on_batch(X)
        # print(y_hat)
        idx = df_useful["idx"].values
        vals = []
        for i in range(len(y_hat)):
            vals.append(y_hat[i][idx[i] - 1])
        return np.mean(vals)

In [329]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) * 24):
        yield start_date + timedelta(hours=n)

In [330]:
def make_prediction(start_date, end_date, model, df):
    pred = {}
    for date in daterange(
        datetime.strptime(start_date, "%d/%m/%Y"),
        datetime.strptime(end_date, "%d/%m/%Y"),
    ):
        val = estimate_price(model, date, df)
        if val != "":
            pred[date.strftime("%Y-%m-%d %H:%M:%S")] = val
    return pred

In [331]:
def evaluate_model(start_date, end_date, model, df_test):
    y_hat = make_prediction(start_date, end_date, model, df_test)
    df_temp = df_test.set_index("created_at")["price"].drop_duplicates()
    mse = 0
    for i in y_hat:
        try:
            mse += (df_temp.loc[i].values[0] - y_hat[i]) ** 2
        except:
            pass
    return mse / len(y_hat)

In [332]:
def make_time_series_dataset(
    crypto, start_month, start_day, end_month, end_day
):
    X = []
    y = []
    try:
        crypt_prices = pd.read_csv(
            f"data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}_minute.csv"
        )

    except:
        crypt_prices = pd.read_csv(
            f"data/raw/crypto/{crypto_dict[crypto]}_2022_{start_month}_{start_day}-2022_{end_month}_{end_day}.csv"
        )

    crypt_prices["time"] = pd.to_datetime(crypt_prices["time"], unit="s")

    crypt_prices_useful = {}

    for i in crypt_prices[["time", "open"]].values:
        if int(i[0].strftime("%M")) == 0 and int(i[0].strftime("%S")) == 0:
            crypt_prices_useful[i[0]] = i[1]
    for i in crypt_prices_useful:
        if (i - timedelta(hours=23)) in crypt_prices_useful:
            X.append(
                [
                    crypt_prices_useful[i]
                    for i in pd.date_range(
                        i - timedelta(hours=23),
                        i - timedelta(hours=1),
                        freq="H",
                    )
                ]
            )
            y.append(crypt_prices_useful[i])
    return X, y

In [333]:
def make_ts_model():
    model = keras.Sequential()
    model.add(keras.layers.LSTM(8, input_shape=(23, 1)))
    model.add(keras.layers.Dense(32))
    model.add(keras.layers.Dense(1))
    model.compile(
        loss="mean_squared_error",
        optimizer="adam",
        metrics=["mean_squared_error"],
    )
    return model

In [334]:
df_avax, X_avax, y_avax = get_dataset("avalanche", "03", "05", "03", "11")
df_avax2, X_avax2, y_avax2 = get_dataset("avalanche", "03", "28", "04", "04")
df_avax3, X_avax3, y_avax3 = get_dataset("avalanche", "03", "11", "03", "12")
df_avax4, X_avax4, y_avax4 = get_dataset("avalanche", "04", "04", "04", "05")

df_avax = df_avax.append(df_avax2)
df_avax = df_avax.append(df_avax3)
df_avax = df_avax.append(df_avax4)

X_avax = X_avax.append(X_avax2)
X_avax = X_avax.append(X_avax3)
X_avax = X_avax.append(X_avax4)

y_avax = y_avax.append(y_avax2)
y_avax = y_avax.append(y_avax3)
y_avax = y_avax.append(y_avax4)

df_avax_test, _, _ = get_dataset("avalanche", "04", "05", "04", "07")

In [335]:
df_btc, X_btc, y_btc = get_dataset("bitcoin", "03", "05", "03", "11")
df_btc2, X_btc2, y_btc2 = get_dataset("bitcoin", "03", "28", "04", "04")
df_btc3, X_btc3, y_btc3 = get_dataset("bitcoin", "03", "11", "03", "12")
df_btc4, X_btc4, y_btc4 = get_dataset("bitcoin", "04", "04", "04", "05")

df_btc = df_btc.append(df_btc2)
df_btc = df_btc.append(df_btc3)
df_btc = df_btc.append(df_btc4)

X_btc = X_btc.append(X_btc2)
X_btc = X_btc.append(X_btc3)
X_btc = X_btc.append(X_btc4)

y_btc = y_btc.append(y_btc2)
y_btc = y_btc.append(y_btc3)
y_btc = y_btc.append(y_btc4)
df_btc_test, _, _ = get_dataset("bitcoin", "04", "05", "04", "07")

In [336]:
df_eth, X_eth, y_eth = get_dataset("ethereum", "03", "05", "03", "11")
df_eth2, X_eth2, y_eth2 = get_dataset("ethereum", "03", "28", "04", "04")
df_eth3, X_eth3, y_eth3 = get_dataset("ethereum", "03", "11", "03", "12")
df_eth4, X_eth4, y_eth4 = get_dataset("ethereum", "04", "04", "04", "05")

df_eth = df_eth.append(df_eth2)
df_eth = df_eth.append(df_eth3)
df_eth = df_eth.append(df_eth4)


X_eth = X_eth.append(X_eth2)
X_eth = X_eth.append(X_eth3)
X_eth = X_eth.append(X_eth4)

y_eth = y_eth.append(y_eth2)
y_eth = y_eth.append(y_eth3)
y_eth = y_eth.append(y_eth4)

df_eth_test, _, _ = get_dataset("ethereum", "04", "05", "04", "07")

In [337]:
df_sol, X_sol, y_sol = get_dataset("solana", "03", "05", "03", "11")
df_sol2, X_sol2, y_sol2 = get_dataset("solana", "03", "28", "04", "04")
df_sol3, X_sol3, y_sol3 = get_dataset("solana", "03", "11", "03", "12")
df_sol4, X_sol4, y_sol4 = get_dataset("solana", "04", "04", "04", "05")

df_sol = df_sol.append(df_sol2)
df_sol = df_sol.append(df_sol3)
df_sol = df_sol.append(df_sol4)


X_sol = X_sol.append(X_sol2)
X_sol = X_sol.append(X_sol3)
X_sol = X_sol.append(X_sol4)

y_sol = y_sol.append(y_sol2)
y_sol = y_sol.append(y_sol3)
y_sol = y_sol.append(y_sol4)

df_sol_test, _, _ = get_dataset("solana", "04", "05", "04", "07")

In [338]:
df_doge, X_doge, y_doge = get_dataset("doge", "03", "05", "03", "11")
df_doge2, X_doge2, y_doge2 = get_dataset("doge", "03", "28", "04", "04")
df_doge3, X_doge3, y_doge3 = get_dataset("doge", "03", "11", "03", "12")
df_doge4, X_doge4, y_doge4 = get_dataset("doge", "04", "04", "04", "05")

df_doge = df_doge.append(df_doge2)
df_doge = df_doge.append(df_doge3)
df_doge = df_doge.append(df_doge4)


X_doge = X_doge.append(X_doge2)
X_doge = X_doge.append(X_doge3)
X_doge = X_doge.append(X_doge4)

y_doge = y_doge.append(y_doge2)
y_doge = y_doge.append(y_doge3)
y_doge = y_doge.append(y_doge4)

df_doge_test, _, _ = get_dataset("doge", "04", "05", "04", "07")

In [375]:
bit_model = make_model(2)
bit_model.fit(X_btc, y_btc, epochs=500, batch_size=10)

Epoch 1/500


ValueError: in user code:

    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/engine/training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/yfreedman/Library/Python/3.7/lib/python/site-packages/keras/engine/

In [340]:
eth_model = make_model()
eth_model.fit(X_eth, y_eth, epochs=1000, batch_size=10)

Epoch 1/1000
141/141 [==============================] - 2s 2ms/step - loss: 10633833.0000 - mean_squared_error: 10633833.0000
Epoch 2/1000
141/141 [==============================] - 0s 3ms/step - loss: 10582520.0000 - mean_squared_error: 10582520.0000
Epoch 3/1000
141/141 [==============================] - 1s 5ms/step - loss: 10475937.0000 - mean_squared_error: 10475937.0000
Epoch 4/1000
141/141 [==============================] - 1s 4ms/step - loss: 10316828.0000 - mean_squared_error: 10316828.0000
Epoch 5/1000
141/141 [==============================] - 1s 4ms/step - loss: 10109908.0000 - mean_squared_error: 10109908.0000
Epoch 6/1000
141/141 [==============================] - 0s 2ms/step - loss: 9859913.0000 - mean_squared_error: 9859913.0000
Epoch 7/1000
141/141 [==============================] - 0s 2ms/step - loss: 9571716.0000 - mean_squared_error: 9571716.0000
Epoch 8/1000
141/141 [==============================] - 0s 2ms/step - loss: 9250239.0000 - mean_squared_error: 9250239.000

In [341]:
doge_model = make_model()
doge_model.fit(X_doge, y_doge, epochs=1000, batch_size=10)

Epoch 1/1000
50/50 [==============================] - 3s 3ms/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 2/1000
50/50 [==============================] - 0s 2ms/step - loss: 4.1098e-04 - mean_squared_error: 4.1098e-04
Epoch 3/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.9835e-04 - mean_squared_error: 1.9835e-04
Epoch 4/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.6983e-04 - mean_squared_error: 1.6983e-04
Epoch 5/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.6080e-04 - mean_squared_error: 1.6080e-04
Epoch 6/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.5256e-04 - mean_squared_error: 1.5256e-04
Epoch 7/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.5161e-04 - mean_squared_error: 1.5161e-04
Epoch 8/1000
50/50 [==============================] - 0s 2ms/step - loss: 1.4412e-04 - mean_squared_error: 1.4412e-04
Epoch 9/1000
50/50 [==============================] - 0s 3ms/ste

In [342]:
sol_model = make_model()
sol_model.fit(X_sol, y_sol, epochs=1000, batch_size=10)

Epoch 1/1000
32/32 [==============================] - 2s 2ms/step - loss: 14326.6631 - mean_squared_error: 14326.6631
Epoch 2/1000
32/32 [==============================] - 0s 3ms/step - loss: 14203.4053 - mean_squared_error: 14203.4053
Epoch 3/1000
32/32 [==============================] - 0s 2ms/step - loss: 13958.5469 - mean_squared_error: 13958.5469
Epoch 4/1000
32/32 [==============================] - 0s 2ms/step - loss: 13380.9980 - mean_squared_error: 13380.9980
Epoch 5/1000
32/32 [==============================] - 0s 2ms/step - loss: 12549.4434 - mean_squared_error: 12549.4434
Epoch 6/1000
32/32 [==============================] - 0s 2ms/step - loss: 11447.4551 - mean_squared_error: 11447.4551
Epoch 7/1000
32/32 [==============================] - 0s 2ms/step - loss: 10124.6475 - mean_squared_error: 10124.6475
Epoch 8/1000
32/32 [==============================] - 0s 2ms/step - loss: 8651.4619 - mean_squared_error: 8651.4619
Epoch 9/1000
32/32 [==============================] - 0s 3

In [343]:
avax_model = make_model()
avax_model.fit(X_avax, y_avax, epochs=1000, batch_size=10)

Epoch 1/1000
5/5 [==============================] - 2s 3ms/step - loss: 6675.2280 - mean_squared_error: 6675.2280
Epoch 2/1000
5/5 [==============================] - 0s 3ms/step - loss: 6667.7134 - mean_squared_error: 6667.7134
Epoch 3/1000
5/5 [==============================] - 0s 3ms/step - loss: 6659.9707 - mean_squared_error: 6659.9707
Epoch 4/1000
5/5 [==============================] - 0s 3ms/step - loss: 6651.4204 - mean_squared_error: 6651.4199
Epoch 5/1000
5/5 [==============================] - 0s 3ms/step - loss: 6642.3164 - mean_squared_error: 6642.3164
Epoch 6/1000
5/5 [==============================] - 0s 3ms/step - loss: 6632.5229 - mean_squared_error: 6632.5229
Epoch 7/1000
5/5 [==============================] - 0s 3ms/step - loss: 6621.7866 - mean_squared_error: 6621.7866
Epoch 8/1000
5/5 [==============================] - 0s 3ms/step - loss: 6610.1597 - mean_squared_error: 6610.1597
Epoch 9/1000
5/5 [==============================] - 0s 2ms/step - loss: 6597.4751 - mean

In [345]:
X_ts_btc, y_ts_btc = make_time_series_dataset(
    "bitcoin", "01", "03", "04", "03"
)
X_test_btc, y_test_btc = make_time_series_dataset(
    "bitcoin", "04", "05", "04", "07"
)
bit_ltsm = make_ts_model()
bit_ltsm.fit(
    X_ts_btc,
    y_ts_btc,
    epochs=500,
    batch_size=10,
    validation_data=(X_test_btc, y_test_btc),
)

Epoch 1/500
214/214 [==============================] - 3s 8ms/step - loss: 1703745792.0000 - mean_squared_error: 1703745792.0000 - val_loss: 1964634240.0000 - val_mean_squared_error: 1964634240.0000
Epoch 2/500
214/214 [==============================] - 1s 6ms/step - loss: 1701958272.0000 - mean_squared_error: 1701958272.0000 - val_loss: 1961746432.0000 - val_mean_squared_error: 1961746432.0000
Epoch 3/500
214/214 [==============================] - 1s 6ms/step - loss: 1698348544.0000 - mean_squared_error: 1698348544.0000 - val_loss: 1956860160.0000 - val_mean_squared_error: 1956860160.0000
Epoch 4/500
214/214 [==============================] - 1s 6ms/step - loss: 1692936064.0000 - mean_squared_error: 1692936064.0000 - val_loss: 1950110976.0000 - val_mean_squared_error: 1950110976.0000
Epoch 5/500
214/214 [==============================] - 1s 6ms/step - loss: 1685883520.0000 - mean_squared_error: 1685883520.0000 - val_loss: 1941685760.0000 - val_mean_squared_error: 1941685760.0000
Epoch

In [344]:
evaluate_model("05/04/2022", "07/04/2022", bit_model, df_btc_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1977856.0811610827

Much better accuracy from sentiment model than just time series

In [346]:
X_ts_eth, y_ts_eth = make_time_series_dataset(
    "ethereum", "01", "03", "04", "03"
)
X_test_eth, y_test_eth = make_time_series_dataset(
    "ethereum", "04", "05", "04", "07"
)
eth_ltsm = make_ts_model()
eth_ltsm.fit(X_ts_eth, y_ts_eth, epochs=500, batch_size=10)

Epoch 1/500
214/214 [==============================] - 3s 7ms/step - loss: 8594297.0000 - mean_squared_error: 8594297.0000
Epoch 2/500
214/214 [==============================] - 1s 7ms/step - loss: 8467160.0000 - mean_squared_error: 8467160.0000
Epoch 3/500
214/214 [==============================] - 2s 7ms/step - loss: 8199233.5000 - mean_squared_error: 8199233.5000
Epoch 4/500
214/214 [==============================] - 2s 7ms/step - loss: 7801239.0000 - mean_squared_error: 7801239.0000
Epoch 5/500
214/214 [==============================] - 2s 7ms/step - loss: 7303241.5000 - mean_squared_error: 7303241.5000
Epoch 6/500
214/214 [==============================] - 2s 9ms/step - loss: 6731117.5000 - mean_squared_error: 6731117.5000
Epoch 7/500
214/214 [==============================] - 2s 10ms/step - loss: 6108516.5000 - mean_squared_error: 6108516.5000
Epoch 8/500
214/214 [==============================] - 2s 9ms/step - loss: 5457400.0000 - mean_squared_error: 5457400.0000
Epoch 9/500
214

In [350]:
evaluate_model("05/04/2022", "07/04/2022", eth_model, df_eth_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


11988.53472818749

Sentiment model significantly outperforms time series

In [347]:
X_ts_avax, y_ts_avax = make_time_series_dataset(
    "avalanche", "01", "03", "04", "03"
)
X_test_avax, y_test_avax = make_time_series_dataset(
    "avalanche", "04", "05", "04", "07"
)

avax_ltsm = make_ts_model()
avax_ltsm.fit(X_ts_avax, y_ts_avax, epochs=500, batch_size=10)

Epoch 1/500
214/214 [==============================] - 3s 6ms/step - loss: 6059.0244 - mean_squared_error: 6059.0239
Epoch 2/500
214/214 [==============================] - 1s 6ms/step - loss: 2668.8311 - mean_squared_error: 2668.8311
Epoch 3/500
214/214 [==============================] - 1s 6ms/step - loss: 183.7445 - mean_squared_error: 183.7445
Epoch 4/500
214/214 [==============================] - 1s 7ms/step - loss: 101.3786 - mean_squared_error: 101.3786
Epoch 5/500
214/214 [==============================] - 1s 6ms/step - loss: 87.3916 - mean_squared_error: 87.3916
Epoch 6/500
214/214 [==============================] - 1s 6ms/step - loss: 68.2191 - mean_squared_error: 68.2191
Epoch 7/500
214/214 [==============================] - 1s 6ms/step - loss: 56.1936 - mean_squared_error: 56.1936
Epoch 8/500
214/214 [==============================] - 1s 6ms/step - loss: 48.8322 - mean_squared_error: 48.8323
Epoch 9/500
214/214 [==============================] - 1s 6ms/step - loss: 43.6500 -

In [351]:
evaluate_model("05/04/2022", "07/04/2022", avax_model, df_avax_test)

0.0

Sentiment is slightly more accurate but the price values are so small here its hard to say one way or another

In [348]:
X_ts_sol, y_ts_sol = make_time_series_dataset("solana", "01", "03", "04", "03")
X_test_sol, y_test_sol = make_time_series_dataset(
    "solana", "04", "05", "04", "07"
)

sol_ltsm = make_ts_model()
sol_ltsm.fit(X_ts_sol, y_ts_sol, epochs=500, batch_size=10)

Epoch 1/500
214/214 [==============================] - 3s 7ms/step - loss: 9314.3174 - mean_squared_error: 9314.3174
Epoch 2/500
214/214 [==============================] - 1s 7ms/step - loss: 3171.2639 - mean_squared_error: 3171.2639
Epoch 3/500
214/214 [==============================] - 1s 6ms/step - loss: 668.0292 - mean_squared_error: 668.0292
Epoch 4/500
214/214 [==============================] - 1s 6ms/step - loss: 470.6090 - mean_squared_error: 470.6090
Epoch 5/500
214/214 [==============================] - 1s 6ms/step - loss: 368.2854 - mean_squared_error: 368.2854
Epoch 6/500
214/214 [==============================] - 1s 6ms/step - loss: 282.5593 - mean_squared_error: 282.5593
Epoch 7/500
214/214 [==============================] - 1s 6ms/step - loss: 228.6641 - mean_squared_error: 228.6641
Epoch 8/500
214/214 [==============================] - 1s 6ms/step - loss: 182.1697 - mean_squared_error: 182.1697
Epoch 9/500
214/214 [==============================] - 1s 6ms/step - loss: 1

In [352]:
evaluate_model("05/04/2022", "07/04/2022", sol_model, df_sol_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1.1206331413501927

Same as above

In [349]:
X_ts_doge, y_ts_doge = make_time_series_dataset("doge", "01", "03", "04", "03")
X_test_doge, y_test_doge = make_time_series_dataset(
    "doge", "04", "05", "04", "07"
)

doge_ltsm = make_ts_model()
doge_ltsm.fit(X_ts_doge, y_ts_doge, epochs=500, batch_size=10)

Epoch 1/500
214/214 [==============================] - 3s 6ms/step - loss: 6.2573e-04 - mean_squared_error: 6.2573e-04
Epoch 2/500
214/214 [==============================] - 2s 7ms/step - loss: 5.0016e-05 - mean_squared_error: 5.0016e-05
Epoch 3/500
214/214 [==============================] - 1s 6ms/step - loss: 1.1370e-05 - mean_squared_error: 1.1370e-05
Epoch 4/500
214/214 [==============================] - 1s 7ms/step - loss: 9.8997e-06 - mean_squared_error: 9.8997e-06
Epoch 5/500
214/214 [==============================] - 1s 6ms/step - loss: 1.0111e-05 - mean_squared_error: 1.0111e-05
Epoch 6/500
214/214 [==============================] - 1s 6ms/step - loss: 9.6376e-06 - mean_squared_error: 9.6376e-06
Epoch 7/500
214/214 [==============================] - 1s 6ms/step - loss: 9.7168e-06 - mean_squared_error: 9.7168e-06
Epoch 8/500
214/214 [==============================] - 1s 6ms/step - loss: 9.9428e-06 - mean_squared_error: 9.9428e-06
Epoch 9/500
214/214 [===========================

In [355]:
evaluate_model("05/04/2022", "07/04/2022", doge_model, df_doge_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1.6238418835871258e-05

Slightly worse but again we're dealing with such small numbers here so idk

Tried making bigger model but not gonna use so ignore the rest of the code

In [364]:
df_tot = df_btc.copy()
df_tot = df_tot.append(df_sol)
df_tot = df_tot.append(df_avax)
df_tot = df_tot.append(df_eth)
df_tot = df_tot.append(df_doge)

X_tot = X_btc.copy()
X_tot = X_tot.append(X_sol)
X_tot = X_tot.append(X_avax)
X_tot = X_tot.append(X_eth)
X_tot = X_tot.append(X_doge)

y_tot = y_btc.copy()
y_tot = y_tot.append(y_sol)
y_tot = y_tot.append(y_avax)
y_tot = y_tot.append(y_eth)
y_tot = y_tot.append(y_doge)

In [372]:
price_df = X_tot["price"]
y_tot = (y_tot.sub(price_df, axis=0)).div(y_tot)
X_tot = X_tot[["authority", "vader_sentiment_compound"]]

In [388]:
tot_model = make_model(2)
tot_model.fit(X_tot, y_tot, batch_size=10, epochs=10)

Epoch 1/10
388/388 [==============================] - 5s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 2/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 3/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 4/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 5/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 6/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 7/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 8/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 9/10
388/388 [==============================] - 1s 3ms/step - loss: nan - mean_squared_error: nan
Epoch 10/10
388/388 [==============================] - 1s 3ms/st